In [1]:
import unicodedata
import requests
import pandas as pd
import duckdb

In [2]:
r = requests.get('https://opendata.infrabel.be/api/explore/v2.1/catalog/datasets/operationele-punten-van-het-netwerk/exports/json?lang=fr&timezone=Europe%2FBerlin')

In [3]:
op_points = r.json()

In [4]:
list(set([row['class_en'] for row in op_points]))

['Junction',
 '6',
 'Service stop',
 'Station',
 'Net borde',
 'Stop in open track',
 'Movable bridge',
 '8',
 'Other',
 'Grid',
 'Connection',
 'Service installation']

In [5]:
df_facilities = pd.read_csv('facilities_flattened.csv')

In [6]:
# Function to remove accents
def remove_accents(text):
    if isinstance(text, str):  # Ensure the input is a string
        # Normalize to NFD (decomposed) form and filter out non-spacing marks
        return ''.join(char for char in unicodedata.normalize('NFD', text) if unicodedata.category(char) != 'Mn')
    return text  # Return the original value if not a string

In [7]:
df_op = pd.DataFrame(op_points)
df_op['commerciallongnamefrench_noaccents'] = df_op['commerciallongnamefrench'].apply(remove_accents)
df_op['commerciallongnamedutch_noaccents'] = df_op['commerciallongnamedutch'].apply(remove_accents)

In [8]:
duckdb.query("SELECT * FROM df_op WHERE commerciallongnamefrench LIKE 'Villers%'").df()

,geo_point_2d,geo_shape,ptcarid,taftapcode,symbolicname,shortnamefrench,shortnamedutch,longnamefrench,longnamedutch,commercialshortnamefrench,commercialshortnamedutch,commercialmiddlenamefrench,commercialmiddlenamedutch,commerciallongnamefrench,commerciallongnamedutch,classification,class_en,class_fr,commerciallongnamefrench_noaccents,commerciallongnamedutch_noaccents
0,"{'lon': 4.533500041466839, 'lat': 50.578090649...","{'type': 'Feature', 'geometry': {'coordinates'...",1189,BE01189,LLE,VILLERS-LA-V,VILLERS-LA-V,VILLERS-LA-VILLE,VILLERS-LA-VILLE,Villers-la-V,Villers-la-V,Villers-la-Ville,Villers-la-Ville,Villers-la-Ville,Villers-la-Ville,Stopplaats,Stop in open track,Point d'arrêt,Villers-la-Ville,Villers-la-Ville


In [9]:
sql = """
    SELECT 
        d.*,
        op.commerciallongnamefrench,
        op.commerciallongnamedutch,
        op.class_en,
        op.taftapcode,
        op.symbolicname
    FROM
        df_facilities d
    LEFT JOIN
        df_op op
        ON d.name ILIKE op.commerciallongnamefrench_noaccents OR d.name ILIKE op.commerciallongnamedutch_noaccents
"""

In [10]:
duckdb.query(sql).df().to_csv('facilities.csv')